In [1]:
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.getOrCreate()


22/11/01 09:19:01 WARN Utils: Your hostname, kevin resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
22/11/01 09:19:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/01 09:19:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Overview of Structured Spark Types


## DataFrames vs Datasets
Dataframes are untyped while datasets are typed. To say that dataframes are untyped is aslightly inaccurate; they have types, but spark maintains them completely and only checks whether those types line up to those specified in the schema at runtime. Datasets, on the other hand, check whether types conform to the specification at compile time. 

## Columns
They represent a simple type like integer or string, a complex type like an array or map, or a null value. Spark tracks all of this type information for you and offers a variety of ways, wicth which you can transform columns

## Rows
A row is nothing but a row of data. We can create these rows manually from sql, from RDDs, from data sources, or manually from scratch

In [2]:
spark.range(2).collect()

[Row(id=0), Row(id=1)]

# Overview of Structured API Execution
We will demonstrate how code is executed across a cluster. Let us walk through the execution of a single structured API query from user code to executed code
* Write DataFrame/Dataset/SQL code
* If valid code, spark converts this to a logical plan
* Spark transforms this logical plan to a physical plan, checking for optimizations along the way
* Spark then executes this physical plan (RDD manipulations) on the cluster

Written code is then submitted to spark either through the console or via a submitted job. This code then passes through the catalyst optimizer, which decides how the code should be executed and lays out a plan for doing so before, finally, the code is run and the result is returned to the user

## Logical Planning
<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/04-Structured API Overview/images/01-logical planning.png">


This logical plan only represents a set of abstract transformations that do not refer to executors or drivers, it's purely to convert the user's set of expressions into the most opptimized version. It does this by converting the user code into an unresolved logical plan. This plan is unresolved because although your code might be valid, the tabl;es or columns that it refers to might not exist. Spark uses the catalog, a repository of all table and dataframe information, to resolve columns and tables in the analyzer. The analyzer might reject the unresolved logical plan if the required table ir column name does not exist in the catalog. If the analyzer can resolve it, the result is passed through the catalyst optimizer, a collection of rules that attempt to optimize the logical plan by pushing down predicates or selections

## Physical Planning
After successfully creating an optimized logical plam, spark then begins the physical planning process. The physical plan, often called a spark plan, specifies how the logical plan will execute in the cluster by generating different physical execution strategies and comparing them through a cost model. An example of the cost comparison might be choosing how to perform  given join by looking at the physical attributes of a given table (how big the table is or how big its partitions are)
<img src="/home/kevin/Desktop/Big-Data-with-Pyspark/04-Structured API Overview/images/03-Physical_plan.png">

Physical planning results ina  serioes of RDDs and transformatio0ns. This result is why you might have heard of spark as a compiler - it takes queries in dataframes, datasets and sql and compiles them into RDD transformations for you

## Execution
Upon selecting a physical plan, spark runs all of this code over RDDs, the lower-level programming interface of spark. Spark performs further optimizations at runtime, generating native java bytecode that can remove entire tasks or stages during execution. Finally, the result is returned to the user

In [3]:
spark.stop()